## Import thư viện


In [28]:
from pyspark.sql.functions import min, max, count
from pyspark.sql.functions import when
from pyspark.sql.functions import year

StatementMeta(sparkpool, 0, 32, Finished, Available, Finished)

## Đọc dữ liệu


In [1]:
%%pyspark
df = spark.read.load('abfss://data@cuoikybigdata.dfs.core.windows.net/part-00000-tid-3970032623210344264-a24d29ab-6531-4fb4-b5ea-0a6c2af054c0-44-1.c000.snappy.parquet', format='parquet')
display(df.limit(10))

StatementMeta(sparkpool, 0, 2, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 2af94ddf-1fc6-4399-88ce-29bc3f96e230)

## Kiểu dữ liệu của các cột trong df


In [2]:
for field in df.schema.fields:
    print(f"Column: {field.name} | Data Type: {field.dataType}")


StatementMeta(sparkpool, 0, 3, Finished, Available, Finished)

Column: id | Data Type: StringType()
Column: longitude | Data Type: DoubleType()
Column: latitude | Data Type: DoubleType()
Column: elevation | Data Type: DoubleType()
Column: title | Data Type: StringType()
Column: place_description | Data Type: StringType()
Column: sig | Data Type: LongType()
Column: mag | Data Type: DoubleType()
Column: magType | Data Type: StringType()
Column: time | Data Type: TimestampType()
Column: updated | Data Type: TimestampType()
Column: sig_class | Data Type: StringType()


## Tổng quan dữ liệu


In [11]:


print("Tổng số sự kiện:", df.count())

# Thời gian sớm nhất và mới nhất của các sự kiện
df.select(min("time").alias("Earliest"), max("time").alias("Latest")).show()

# Thời gian cập nhật sớm nhất và mới nhất
df.select(min("updated").alias("Cập nhật sớm nhất"), max("updated").alias("Cập nhật mới nhất")).show()


StatementMeta(sparkpool, 0, 12, Finished, Available, Finished)

Tổng số sự kiện: 818243
+--------------------+--------------------+
|            Earliest|              Latest|
+--------------------+--------------------+
|2020-01-01 00:01:...|2025-04-29 23:54:...|
+--------------------+--------------------+

+--------------------+--------------------+
|   Cập nhật sớm nhất|   Cập nhật mới nhất|
+--------------------+--------------------+
|2020-01-01 00:28:...|2025-05-24 13:57:...|
+--------------------+--------------------+



## Độ lớn của động đất


In [29]:

mag_types = df.select("magType").distinct().rdd.flatMap(lambda x: x).collect()

print("Danh sách các kiểu độ lớn (magType) có trong dữ liệu:")
for mag in mag_types:
    print(mag)

# Phân phối số lượng theo từng loại độ lớn (magType)
display(df.groupBy("magType").count().orderBy("count", ascending=False))


StatementMeta(sparkpool, 0, 33, Finished, Available, Finished)

Danh sách các kiểu độ lớn (magType) có trong dữ liệu:
ms_20
mb_lg
mlg
None
mlr
ml(texnet)
mlv
mww
mwr
ml
mwb
ms_vx
mh
mb
mwp
md
mwc
mun
mw


SynapseWidget(Synapse.DataFrame, 93abf7aa-70fd-4ea4-a13b-235eebf8f22d)

## Phân tích theo vị trí địa lý


In [15]:
# Top 10 khu vực có nhiều sự kiện động đất nhất
display(df.groupBy("place_description").count().orderBy("count", ascending=False).limit(10))


StatementMeta(sparkpool, 0, 19, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 4465aa48-db23-4f83-a25e-05c854ce5d4c)

## Phân tích theo độ cao và địa hình


In [27]:

# Phân loại độ cao
df_with_elevation = df.withColumn(
    "elevation_class",
    when(df.elevation < 0, "Sea")
    .when((df.elevation >= 0) & (df.elevation < 200), "Coastal lowland")
    .when((df.elevation >= 200) & (df.elevation < 500), "Inland lowland")
    .when((df.elevation >= 500) & (df.elevation < 1000), "Upland")
    .when((df.elevation >= 1000) & (df.elevation < 3000), "Mountain")
    .otherwise("High mountain")
)


# Số lượng động đất theo loại địa hình
display(df_with_elevation.groupBy("elevation_class").count().orderBy("count", ascending=False))


StatementMeta(sparkpool, 0, 31, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 3b584486-a4c9-4016-b882-ce4348aba626)

##


## Phân tích độ nghiêm trọng


In [22]:
# Số lượng sự kiện theo từng nhóm phân loại độ nghiêm trọng (sig_class)
display(df.groupBy("sig_class").count().orderBy("count", ascending=False))



StatementMeta(sparkpool, 0, 26, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, ba0f8839-80bc-445b-99e4-00975840780e)

In [23]:
# Top 5 khu vực có mức độ nghiêm trọng cao nhất
display(df.orderBy("sig", ascending=False).select("title", "sig", "mag", "time", "place_description").limit(5))


StatementMeta(sparkpool, 0, 27, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 51d410db-cf71-4dc8-8234-da4f3b4a79a7)

## Phân tích theo thời gian


In [24]:


# Tạo cột mới 'year' từ cột thời gian
df_with_year = df.withColumn("year", year("time"))

# Số động đất theo từng năm
display(df_with_year.groupBy("year").count().orderBy("year"))


StatementMeta(sparkpool, 0, 28, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 4d666790-5c43-42c5-b9da-658366297146)

## Mối quan hệ giữa độ lớn và độ nghiêm trọng


In [30]:
from pyspark.sql.functions import corr

# Tính hệ số tương quan giữa độ lớn và độ nghiêm trọng
df.select(corr("mag", "sig")).show()


StatementMeta(sparkpool, 0, 34, Finished, Available, Finished)

+------------------+
|    corr(mag, sig)|
+------------------+
|0.9197624785628857|
+------------------+



In [32]:
#Hiển thị dữ liệu mẫu thể hiện mối liên hệ giữa mag và sig
display(
    df.select("mag", "sig")
      .orderBy("mag", ascending=False)
      .limit(10)
)


StatementMeta(sparkpool, 0, 36, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, fcb979fa-c9c0-475b-aed7-b2c20eec61eb)

In [33]:

df_scatter = df.select("mag", "sig").dropna()

display(df_scatter)


StatementMeta(sparkpool, 0, 37, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 913dc232-8c66-4063-8ce0-216ff6fb65b2)

## Kết luận


### Kết luận tổng thể về dữ liệu động đất:

### Phân bố kiểu độ lớn (magType):

##### Kiểu ml (local magnitude) chiếm đa số, cho thấy phần lớn các trận động đất được ghi nhận là các sự kiện nhỏ, gần tâm chấn, được mạng lưới địa chấn địa phương theo dõi.

##### Các kiểu khác như md, mb cũng có tỷ lệ đáng kể, nhưng các kiểu còn lại rất ít xuất hiện, thể hiện sự đa dạng trong cách đo lường tùy theo tính chất trận động đất (quy mô, độ sâu, loại sóng).

### Phân bố địa lý các trận động đất:

##### Các vùng hoạt động địa chấn mạnh bao gồm California (The Geysers), Alaska, Hawaii, South Sandwich Islands, đều thuộc các vùng ranh giới mảng kiến tạo hoặc vùng núi lửa, đứt gãy địa chất lớn.

##### Khu vực ven biển (coastal lowland) chiếm phần lớn số sự kiện (hơn 750,000), phản ánh đặc điểm địa chất nơi có hoạt động kiến tạo mạnh.

##### Các vùng dưới biển (sea) cũng ghi nhận nhiều trận động đất, phù hợp với rãnh đại dương và hoạt động kiến tạo dưới đáy biển.

##### Các vùng nội địa và vùng núi ít xảy ra động đất hơn, trừ khi gần đứt gãy địa chất.

### Phân loại độ nghiêm trọng (sig):

### Phần lớn trận động đất (~83%) thuộc nhóm Low về độ nghiêm trọng, tức thường là các trận động đất nhẹ, ít gây thiệt hại.

##### Khoảng 16% là nhóm Moderate, có thể gây ảnh hưởng cục bộ nhưng không nghiêm trọng trên diện rộng.

##### Nhóm High rất hiếm (~0.2%), nhưng lại là những sự kiện có khả năng gây hậu quả lớn về người và tài sản, cần đặc biệt quan tâm.

##### Việc phân nhóm này giúp ưu tiên phân bổ nguồn lực quản lý rủi ro thiên tai hiệu quả.

### Các trận động đất nghiêm trọng nổi bật:

##### Những trận như M7.8 tại Thổ Nhĩ Kỳ, M7.7 tại Myanmar (2025), M7.1 tại Tây Tạng có chỉ số độ nghiêm trọng sig rất cao, phản ánh mức độ ảnh hưởng lớn.

##### Chỉ số sig không chỉ phụ thuộc vào độ lớn mà còn liên quan mật thiết đến vị trí xảy ra (mật độ dân cư, hạ tầng), thiệt hại thực tế.

##### Chuỗi động đất tại khu vực Kahramanmaras (Thổ Nhĩ Kỳ) là điểm nóng với ảnh hưởng nghiêm trọng đặc biệt cần chú ý.

### Xu hướng số lượng trận động đất theo năm:

##### Số vụ động đất đạt đỉnh vào năm 2020 (~175,000 vụ), sau đó có xu hướng giảm nhẹ và ổn định trong giai đoạn 2021-2023.

##### Năm 2024 ghi nhận tăng nhẹ.

##### Xu hướng này có thể phản ánh thay đổi tự nhiên hoặc ảnh hưởng của hệ thống ghi nhận, cần theo dõi thêm.

### Mối quan hệ giữa độ lớn (mag) và độ nghiêm trọng (sig):

##### Có xu hướng chung là khi độ nghiêm trọng tăng, độ lớn cũng có khả năng cao hơn, nhưng không phải quy luật chặt chẽ.

##### Nhiều trường hợp độ nghiêm trọng cao nhưng độ lớn lại nhỏ, cho thấy yếu tố vị trí, mật độ dân cư và thiệt hại ảnh hưởng lớn đến đánh giá nghiêm trọng.

### Tóm tắt ý nghĩa thực tiễn:

##### Động đất chủ yếu xảy ra tại vùng ven biển và vùng có hoạt động kiến tạo mạnh, đặc biệt là California, Alaska, Hawaii và các rãnh đại dương.

##### Mặc dù đa phần là các trận động đất nhẹ, không gây thiệt hại đáng kể, nhưng những trận nghiêm trọng dù ít nhưng cần được cảnh báo và quản lý nghiêm túc.

##### Các yếu tố ngoài độ lớn như vị trí địa lý và mật độ dân cư đóng vai trò quyết định trong mức độ nghiêm trọng của động đất.

##### Việc phân loại và phân tích chi tiết theo magType, địa điểm và mức độ nghiêm trọng giúp cải thiện công tác dự báo, giảm thiểu thiệt hại và tối ưu nguồn lực ứng phó.
